In [17]:
import pandas as pd
from sklearn import model_selection
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
print('INFO: imports ready!')

INFO: imports ready!


In [ ]:
#!giskard worker start -d

In [18]:
url = 'https://raw.githubusercontent.com/Giskard-AI/examples/main/datasets/credit_scoring_classification_model_dataset/german_credit_prepared.csv'
credit = pd.read_csv(url, sep=',',engine="python")

In [19]:
# Declare the type of each column in the dataset(example: category, numeric, text)
column_types = {'default':"category",
               'account_check_status':"category",
               'duration_in_month':"numeric",
               'credit_history':"category",
               'purpose':"category",
               'credit_amount':"numeric",
               'savings':"category",
               'present_employment_since':"category",
               'installment_as_income_perc':"numeric",
               'sex':"category",
               'personal_status':"category",
               'other_debtors':"category",
               'present_residence_since':"numeric",
               'property':"category",
               'age':"numeric",
               'other_installment_plans':"category",
               'housing':"category",
               'credits_this_bank':"numeric",
               'job':"category",
               'people_under_maintenance':"numeric",
               'telephone':"category",
               'foreign_worker':"category"}

In [26]:
# feature_types is used to declare the features the model is trained on
feature_types = {i:column_types[i] for i in column_types if i!='default'}

# Pipeline to fill missing values, transform and scale the numeric columns
columns_to_scale = [key for key in feature_types.keys() if feature_types[key]=="numeric"]
numeric_transformer = Pipeline([('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])

# Pipeline to fill missing values and one hot encode the categorical values
columns_to_encode = [key for key in feature_types.keys() if feature_types[key]=="category"]
categorical_transformer = Pipeline([
        ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
        ('onehot', OneHotEncoder(handle_unknown='ignore',sparse_output=False)) ])

# Perform preprocessing of the columns with the above pipelines
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, columns_to_scale),
      ('cat', categorical_transformer, columns_to_encode)
          ]
)

# Pipeline for the model Logistic Regression
clf_logistic_regression = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', LogisticRegression(max_iter =1000))])

# Split the data into train and test
Y=credit['default']
X= credit.drop(columns="default")
X_train, X_test, Y_train, Y_test = model_selection.train_test_split(X, Y, test_size=0.20,random_state = 30, stratify = Y)

In [32]:
# Fit and score your model
clf_logistic_regression.fit(X_train, Y_train)

print(f"Accuracy whe have to beat: {clf_logistic_regression.score(X_test, Y_test)}%")
print('Model description')
clf_logistic_regression

Accuracy whe have to beat: 0.755%
Model description


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['duration_in_month',
                                                   'credit_amount',
                                                   'installment_as_income_perc',
                                                   'present_residence_since',
                                                   'age', 'credits_this_bank',
                                                   'people_under_maintenance']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleI...
                                                                                 strategy='constant')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse_output=False))]),
                                                  ['account_check_status',
                                                   'credit_history', 'purpose',
                                                   'savings',
                                                   'present_employment_since',
                                                   'sex', 'personal_status',
                                                   'other_debtors', 'property',
                                                   'other_installment_plans',
                                                   'housing', 'job',
                                                   'telephone',
                                                   'foreign_worker'])])),
                ('classifier', LogisticRegression(max_iter=1000))])

### Data exploration

In [45]:
for col in credit.columns:
    print(f'-----Details for column: "{col}"-----')
    print(credit[col].value_counts(sort=True, dropna = False))


-----Details for column: "default"-----
Not default    700
Default        300
Name: default, dtype: int64
-----Details for column: "account_check_status"-----
no checking account                                   394
< 0 DM                                                274
0 <= ... < 200 DM                                     269
>= 200 DM / salary assignments for at least 1 year     63
Name: account_check_status, dtype: int64
-----Details for column: "duration_in_month"-----
24    184
12    179
18    113
36     83
6      75
15     64
9      49
48     48
30     40
21     30
10     28
60     13
27     13
42     11
11      9
20      8
8       7
4       6
45      5
7       5
39      5
14      4
13      4
33      3
28      3
54      2
16      2
22      2
47      1
5       1
26      1
72      1
40      1
Name: duration_in_month, dtype: int64
-----Details for column: "credit_history"-----
existing credits paid back duly till now                       530
critical account/ other credits exis

# Ideas!

1) How do I find "slices" or groups of the dataset that "underperform" ----> investigate this first
2) Use K-modes to cluster and focus on the over sampling techniques in certain clusters
3) Use naive-bayes to fill the missing values
4) Look at feature importance and correlation between features
5) Feature augmentation?
6) Look on "how to see what features are the hardest to classify" or the ones that are giving trouble to the model
7) Correlation between missed/wrong detections and the features
8) Do the oversampling using a fancy method from imbalanced learn